In [59]:
# imports

import pandas as pd
import os
import ast
import numpy as np
import tensorflow as tf
import einops

from keras.layers import Input, Dense, concatenate, Embedding, ReLU, Dropout, Bidirectional, GRU, Add, Concatenate
from keras import Model

In [60]:
# constants

UNITS = 256

BUFFER_SIZE = 120
BATCH_SIZE = 64

max_name_features = 5000
max_name_len = 4

max_ingredient_features = 2000
max_ingredient_len = 4

max_step_features = 10000
max_step_len = 400

## Load data from file

In [61]:
text_path = os.path.join('data', 'choc_recipes.txt')
# Using readlines()
file1 = open(text_path, 'r')
Lines = file1.readlines()

names = []
ingredients = []
steps = []

count = 0
# Strips the newline character
for line in Lines:
    pieces = line.split('\t')
    names.append(pieces[0])
    ingredients.append(pieces[1])
    steps.append(pieces[2])

In [62]:
ingredients_list_list = []
for i_string in ingredients:
    i = i_string.replace(', ', '*')
    i = i.replace(' ', '_')
    i = i.replace('*', ', ')

    ingredients_list_list.append(i)

ingredients = ingredients_list_list

In [63]:
print(f'Name count: {len(names)}')
print(f'Ingredient count: {len(ingredients)}')
print(f'Step count: {len(steps)}')

Name count: 7404
Ingredient count: 7404
Step count: 7404


## Create Tensor Flow Datasets

In [64]:
names = np.array(names)
ingredients = np.array(ingredients)
steps = np.array(steps)

In [65]:
BUFFER_SIZE = len(names)
BATCH_SIZE = 64

is_train = np.random.uniform(size=(len(names),)) < 0.8

# train
train_dataset = tf.data.Dataset.from_tensor_slices(({"names": names[is_train], "ingredients": ingredients[is_train]}, steps[is_train]))
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

# test
test_dataset = tf.data.Dataset.from_tensor_slices(({"names": names[~is_train], "ingredients": ingredients[~is_train]}, steps[~is_train]))
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)

## Create Text Vectorizer

In [66]:
step_vectorizer = tf.keras.layers.TextVectorization(
 max_tokens=max_step_features,
 output_mode='int',
 output_sequence_length=max_step_len)

#train_raw.map(lambda context, target: context)
step_vectorizer.adapt(steps)

In [67]:
step_vectorizer.get_vocabulary()[:10]

['', '[UNK]', 'step', 'and', 'the', 'in', 'a', 'to', 'until', 'with']

In [68]:
ingredient_vectorizer = tf.keras.layers.TextVectorization(
 max_tokens=max_ingredient_features,
 output_mode='int',
 output_sequence_length=max_ingredient_len)

#train_raw.map(lambda context, target: context)

ingredient_vectorizer.adapt(ingredients)

In [69]:
ingredient_vectorizer.get_vocabulary()[:10]

['',
 '[UNK]',
 'butter',
 'sugar',
 'salt',
 'eggs',
 'vanilla',
 'bakingsoda',
 'vanillaextract',
 'flour']

In [70]:
name_vectorizer = tf.keras.layers.TextVectorization(
 max_tokens=max_name_features,
 output_mode='int',
 output_sequence_length=max_name_len)

#train_raw.map(lambda context, target: context)
name_vectorizer.adapt(names)

In [71]:
name_vectorizer.get_vocabulary()[:10]

['',
 '[UNK]',
 'chocolate',
 'cake',
 'cookies',
 'chip',
 's',
 'brownies',
 'fudge',
 'butter']

In [72]:
def process_text(inputs, outputs):

  names = inputs['names']
  ingredients = inputs['ingredients']

  name_context = name_vectorizer(names)
  ingredient_context = ingredient_vectorizer(ingredients)
  context = {'names': name_context, 'ingredients': ingredient_context}

  target = step_vectorizer(outputs)
  targ_in = target[:,:-1]
  targ_out = target[:,1:]
  return (context, targ_in), targ_out


train_ds = train_dataset.map(process_text, tf.data.AUTOTUNE)
test_ds = test_dataset.map(process_text, tf.data.AUTOTUNE)

In [73]:
for (ex_context_tok, ex_tar_in), ex_tar_out in train_ds.take(1):
  
  name_tok = ex_context_tok['names']
  ingredient_tok = ex_context_tok['ingredients']
  print(name_tok[0, :10].numpy()) 
  print(ingredient_tok[0, :10].numpy()) 
  print()

  print(ex_tar_in[0, :10].numpy()) 
  print(ex_tar_out[0, :10].numpy())

[ 975   33 1876    7]
[2 3 6 5]

[ 54  30   7  68  77   2 128 304 217 190]
[ 30   7  68  77   2 128 304 217 190  28]


## Encoder

In [74]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, name_processor, ingredient_processor, units):
    super(Encoder, self).__init__()
    
    self.name_processor = name_processor
    self.ingredient_processor = ingredient_processor

    self.name_vocab_size = name_processor.vocabulary_size()
    self.ingredient_vocab_size = ingredient_processor.vocabulary_size()
    
    self.units = units

    # The embedding layer converts tokens to vectors
    self.name_embedding = tf.keras.layers.Embedding(self.name_vocab_size, units, mask_zero=True)
    self.ingredient_embedding = tf.keras.layers.Embedding(self.ingredient_vocab_size, units, mask_zero=True)

    self.concat = Concatenate(axis=-1)

    self.embedding = tf.keras.layers.Embedding((self.name_vocab_size + self.ingredient_vocab_size), units, mask_zero=True)
    #self.embedding = tf.keras.layers.Embedding(self.name_vocab_size, units, mask_zero=True)

    # The RNN layer processes those vectors sequentially.
    self.rnn = tf.keras.layers.Bidirectional(
        merge_mode='sum',
        layer=tf.keras.layers.GRU(units,
                            # Return the sequence and state
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform'))

  def call(self, x):


    names = x['names']
    ingredients = x['ingredients']

    #shape_checker = ShapeChecker()
    #shape_checker(x, 'batch s')

    # 2. The embedding layer looks up the embedding vector for each token.
    x1 = self.name_embedding(names)
    print(f'Dimensions of x1: {x1.shape}')
    x2 = self.ingredient_embedding(ingredients)
    print(f'Dimensions of x2: {x2.shape}')
    x = self.concat([x1, x2])
    print(f'Dimensions of x after concat: {x.shape}')


    #print(f'Dimensions of x after concat: {x.shape}')
    #x = x1
    #x = self.embedding(x)

    #shape_checker(x, 'batch s units')
    
   
   
    # 3. The GRU processes the sequence of embeddings.
    x = self.rnn(x)
    print(f'Dimensions of x after rnn: {x.shape}')
    #shape_checker(x, 'batch s units')
  
    # 4. Returns the new sequence of embeddings.
    return x

  def convert_input(self, ingredients, names):

    names = self.name_processor(names)#.to_tensor()
    print(f'Name Tokens: {names}')
    
    ingredients = self.ingredient_processor(ingredients)#.to_tensor()
    print(f'Ingredients Tokens: {ingredients}')
    x = {"names": names, "ingredients": ingredients}
    context = self(x)
    return context

In [75]:
# Encode the input sequence.
encoder = Encoder(name_vectorizer, ingredient_vectorizer, UNITS)
ex_context = encoder(ex_context_tok)

#print(f'Context tokens, shape (batch, s): {ex_context_tok.shape}')
print(f'Encoder output, shape (batch, s, units): {ex_context.shape}')

Dimensions of x1: (64, 4, 256)
Dimensions of x2: (64, 4, 256)
Dimensions of x after concat: (64, 4, 512)
Dimensions of x after rnn: (64, 4, 256)
Encoder output, shape (batch, s, units): (64, 4, 256)


In [76]:
result = encoder.convert_input(['sugar, salt, eggs, vanilla'], ['chocolate brownies'])
print(result)

Name Tokens: [[2 7 0 0]]
Ingredients Tokens: [[3 4 5 6]]
Dimensions of x1: (1, 4, 256)
Dimensions of x2: (1, 4, 256)
Dimensions of x after concat: (1, 4, 512)
Dimensions of x after rnn: (1, 4, 256)
tf.Tensor(
[[[-0.01905754 -0.01716275 -0.00262569 ...  0.01229564  0.01274323
   -0.01800387]
  [-0.02524667  0.01264304  0.00223324 ... -0.00802431  0.02874554
   -0.01396075]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]], shape=(1, 4, 256), dtype=float32)


## CrossAttention

In [77]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

  def call(self, x, context):
    #shape_checker = ShapeChecker()

    #shape_checker(x, 'batch t units')
    #shape_checker(context, 'batch s units')

    attn_output, attn_scores = self.mha(
        query=x,
        value=context,
        return_attention_scores=True)

    #shape_checker(x, 'batch t units')
    #shape_checker(attn_scores, 'batch heads t s')

    # Cache the attention scores for plotting later.
    attn_scores = tf.reduce_mean(attn_scores, axis=1)
    #shape_checker(attn_scores, 'batch t s')
    self.last_attention_weights = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [78]:
attention_layer = CrossAttention(UNITS)

# Attend to the encoded tokens
embed = tf.keras.layers.Embedding(step_vectorizer.vocabulary_size(),
                                  output_dim=UNITS, mask_zero=True)
ex_tar_embed = embed(ex_tar_in)

result = attention_layer(ex_tar_embed, ex_context)

print(f'Context sequence, shape (batch, s, units): {ex_context.shape}')
print(f'Target sequence, shape (batch, t, units): {ex_tar_embed.shape}')
print(f'Attention result, shape (batch, t, units): {result.shape}')
print(f'Attention weights, shape (batch, t, s):    {attention_layer.last_attention_weights.shape}')

Context sequence, shape (batch, s, units): (64, 4, 256)
Target sequence, shape (batch, t, units): (64, 399, 256)
Attention result, shape (batch, t, units): (64, 399, 256)
Attention weights, shape (batch, t, s):    (64, 399, 4)


In [79]:
attention_layer.last_attention_weights[0].numpy().sum(axis=-1)

array([1.        , 1.        , 1.0000001 , 1.        , 0.99999994,
       1.0000001 , 1.0000001 , 1.0000001 , 1.0000001 , 1.        ,
       0.99999994, 1.        , 1.0000001 , 0.9999999 , 1.        ,
       1.        , 1.        , 0.99999994, 1.        , 1.0000001 ,
       1.        , 0.99999994, 1.        , 1.0000001 , 0.9999999 ,
       1.        , 0.99999994, 1.        , 1.        , 0.99999994,
       1.0000001 , 1.0000001 , 1.        , 1.        , 1.        ,
       0.99999994, 1.        , 0.99999994, 1.0000001 , 1.        ,
       1.        , 1.0000001 , 1.        , 1.        , 1.        ,
       1.0000001 , 1.        , 0.99999994, 0.99999994, 1.        ,
       1.0000001 , 1.0000001 , 1.        , 1.        , 0.9999999 ,
       1.        , 1.        , 1.0000001 , 1.        , 1.0000001 ,
       1.        , 0.9999999 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.99999994, 0.99999994, 1.        ,
       0.99999994, 1.        , 1.0000001 , 1.        , 1.00000

## Decoder

In [80]:
class Decoder(tf.keras.layers.Layer):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, text_processor, units):
    super(Decoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.word_to_id = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]')
    self.id_to_word = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]',
        invert=True)
    self.start_token = self.word_to_id('[START]')
    self.end_token = self.word_to_id('[END]')

    self.units = units


    # 1. The embedding layer converts token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size,
                                               units, mask_zero=True)

    # 2. The RNN keeps track of what's been generated so far.
    self.rnn = tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    # 3. The RNN output will be the query for the attention layer.
    self.attention = CrossAttention(units)

    # 4. This fully connected layer produces the logits for each
    # output token.
    self.output_layer = tf.keras.layers.Dense(self.vocab_size)

In [81]:
@Decoder.add_method
def call(self,
         context, x,
         state=None,
         return_state=False):  
  #shape_checker = ShapeChecker()
  #shape_checker(x, 'batch t')
  #shape_checker(context, 'batch s units')

  # 1. Lookup the embeddings
  x = self.embedding(x)
  #shape_checker(x, 'batch t units')

  # 2. Process the target sequence.
  x, state = self.rnn(x, initial_state=state)
  #shape_checker(x, 'batch t units')

  # 3. Use the RNN output as the query for the attention over the context.
  x = self.attention(x, context)
  self.last_attention_weights = self.attention.last_attention_weights
  #shape_checker(x, 'batch t units')
  #shape_checker(self.last_attention_weights, 'batch t s')

  # Step 4. Generate logit predictions for the next token.
  logits = self.output_layer(x)
  #shape_checker(logits, 'batch t target_vocab_size')

  if return_state:
    return logits, state
  else:
    return logits

In [82]:
decoder = Decoder(step_vectorizer, UNITS)

In [83]:
logits = decoder(ex_context, ex_tar_in)

print(f'encoder output shape: (batch, s, units) {ex_context.shape}')
print(f'input target tokens shape: (batch, t) {ex_tar_in.shape}')
print(f'logits shape shape: (batch, target_vocabulary_size) {logits.shape}')

encoder output shape: (batch, s, units) (64, 4, 256)
input target tokens shape: (batch, t) (64, 399)
logits shape shape: (batch, target_vocabulary_size) (64, 399, 8257)


In [84]:
@Decoder.add_method
def get_initial_state(self, context):
  batch_size = tf.shape(context)[0]
  start_tokens = tf.fill([batch_size, 1], self.start_token)
  done = tf.zeros([batch_size, 1], dtype=tf.bool)
  embedded = self.embedding(start_tokens)
  return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

In [85]:
@Decoder.add_method
def tokens_to_text(self, tokens):
  words = self.id_to_word(tokens)
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
  result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
  return result

In [86]:
@Decoder.add_method
def get_next_token(self, context, next_token, done, state, temperature = 0.0):
  logits, state = self(
    context, next_token,
    state = state,
    return_state=True) 

  if temperature == 0.0:
    next_token = tf.argmax(logits, axis=-1)
  else:
    logits = logits[:, -1, :]/temperature
    next_token = tf.random.categorical(logits, num_samples=1)

  # If a sequence produces an `end_token`, set it `done`
  done = done | (next_token == self.end_token)
  # Once a sequence is done it only produces 0-padding.
  next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

  return next_token, done, state

In [87]:
# Setup the loop variables.
next_token, done, state = decoder.get_initial_state(ex_context)
tokens = []

for n in range(10):
  # Run one step.
  next_token, done, state = decoder.get_next_token(
      ex_context, next_token, done, state, temperature=1.0)
  # Add the token to the output.
  tokens.append(next_token)

# Stack all the tokens together.
tokens = tf.concat(tokens, axis=-1) # (batch, t)

# Convert the tokens back to a a string
result = decoder.tokens_to_text(tokens)
result[:3].numpy()

array([b'toss dropfuls poke chocoalte cigar extinguished mollases decided length timed',
       b'having donuts microwaveble child ever preferably fluted fahrenheit between mushroom',
       b'shortcakes dietary out genatin liqeuer roasters wide intense youll exchanges'],
      dtype=object)

## Translator

In [88]:
class Translator(tf.keras.Model):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, units,
               name_vectorizer,
               ingredient_vectorizer,
               step_vectorizer):
    super().__init__()
    # Build the encoder and decoder
    encoder = Encoder(name_vectorizer, ingredient_vectorizer, units)
    decoder = Decoder(step_vectorizer, units)

    self.encoder = encoder
    self.decoder = decoder

  def call(self, inputs):
    context, x = inputs
    context = self.encoder(context)
    logits = self.decoder(context, x)

    #TODO(b/250038731): remove this
    try:
      # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
      del logits._keras_mask
    except AttributeError:
      pass

    return logits

In [89]:
model = Translator(UNITS, name_vectorizer, ingredient_vectorizer, step_vectorizer)

logits = model((ex_context_tok, ex_tar_in))

#print(f'Context tokens, shape: (batch, s, units) {ex_context_tok.shape}')
print(f'Target tokens, shape: (batch, t) {ex_tar_in.shape}')
print(f'logits, shape: (batch, t, target_vocabulary_size) {logits.shape}')

Dimensions of x1: (64, 4, 256)
Dimensions of x2: (64, 4, 256)
Dimensions of x after concat: (64, 4, 512)
Dimensions of x after rnn: (64, 4, 256)
Target tokens, shape: (batch, t) (64, 399)
logits, shape: (batch, t, target_vocabulary_size) (64, 399, 8257)


In [90]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [91]:
def masked_acc(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [92]:
model.compile(optimizer='adam',
              loss=masked_loss, 
              metrics=[masked_acc, masked_loss])

In [93]:
vocab_size = 1.0 * step_vectorizer.vocabulary_size()

{"expected_loss": tf.math.log(vocab_size).numpy(),
 "expected_acc": 1/vocab_size}

{'expected_loss': 9.018817, 'expected_acc': 0.00012110936175366356}

In [94]:
model.evaluate(test_ds, steps=20, return_dict=True)

Dimensions of x1: (None, 4, 256)
Dimensions of x2: (None, 4, 256)
Dimensions of x after concat: (None, 4, 512)
Dimensions of x after rnn: (None, 4, 256)
Dimensions of x1: (None, 4, 256)
Dimensions of x2: (None, 4, 256)
Dimensions of x after concat: (None, 4, 512)
Dimensions of x after rnn: (None, 4, 256)
20/20 [==============================] - 77s 3s/step - loss: 9.0196 - masked_acc: 7.3931e-05 - masked_loss: 9.0196


{'loss': 9.019614219665527,
 'masked_acc': 7.393107807729393e-05,
 'masked_loss': 9.019614219665527}

In [95]:
history = model.fit(
    train_ds.repeat(), 
    epochs=1,
    steps_per_epoch = 100,
    validation_data=test_ds,
    validation_steps = 20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3)])

Dimensions of x1: (None, 4, 256)
Dimensions of x2: (None, 4, 256)
Dimensions of x after concat: (None, 4, 512)
Dimensions of x after rnn: (None, 4, 256)
Dimensions of x1: (None, 4, 256)
Dimensions of x2: (None, 4, 256)
Dimensions of x after concat: (None, 4, 512)
Dimensions of x after rnn: (None, 4, 256)
100/100 [==============================] - 613s 6s/step - loss: 5.3349 - masked_acc: 0.1496 - masked_loss: 5.3349 - val_loss: 3.8811 - val_masked_acc: 0.2896 - val_masked_loss: 3.8811


## Rezepte generieren

In [96]:
@Translator.add_method
def translate(self, ingredients, names, *, max_length=50, temperature=0.0):
  context = self.encoder.convert_input(ingredients, names)
  batch_size = tf.shape(names)[0]
  
  # Setup the loop inputs
  tokens = []
  attention_weights = []
  next_token, done, state = self.decoder.get_initial_state(context)

  for _ in range(max_length):
    # Generate the next token
    next_token, done, state = self.decoder.get_next_token(
        context, next_token, done,  state, temperature)

    # Collect the generated tokens
    tokens.append(next_token)
    attention_weights.append(self.decoder.last_attention_weights)

    if tf.executing_eagerly() and tf.reduce_all(done):
      break

  # Stack the lists of tokens and attention weights.
  tokens = tf.concat(tokens, axis=-1)   # t*[(batch 1)] -> (batch, t)
  self.last_attention_weights = tf.concat(attention_weights, axis=1)  # t*[(batch 1 s)] -> (batch, t s)

  result = self.decoder.tokens_to_text(tokens)
  return result

In [97]:
result = model.translate(['sugar, salt, eggs, vanilla'], ['chocolate brownies'])
result[0].numpy().decode()

Name Tokens: [[2 7 0 0]]
Ingredients Tokens: [[3 4 5 6]]
Dimensions of x1: (1, 4, 256)
Dimensions of x2: (1, 4, 256)
Dimensions of x after concat: (1, 4, 512)
Dimensions of x after rnn: (1, 4, 256)


'to cool step add the chocolate chips and vanilla step add the chocolate chips and vanilla step add the chocolate chips and vanilla step add the chocolate chips and vanilla step add the chocolate chips and vanilla step add the chocolate chips and vanilla step add the chocolate chips and'

## Save Model

In [98]:
class Export(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None]), tf.TensorSpec(dtype=tf.string, shape=[None])])
  def translate(self, ingredients, names):
    return self.model.translate(ingredients, names)

export = Export(model)

tf.saved_model.save(export, 'translator', signatures={'serving_default': export.translate})

Name Tokens: Tensor("text_vectorization_5/RaggedToTensor/RaggedTensorToTensor:0", shape=(None, 4), dtype=int64)
Ingredients Tokens: Tensor("text_vectorization_4/RaggedToTensor/RaggedTensorToTensor:0", shape=(None, 4), dtype=int64)
Dimensions of x1: (None, 4, 256)
Dimensions of x2: (None, 4, 256)
Dimensions of x after concat: (None, 4, 512)
Dimensions of x after rnn: (None, 4, 256)
Dimensions of x1: (None, 4, 256)
Dimensions of x2: (None, 4, 256)
Dimensions of x after concat: (None, 4, 512)
Dimensions of x after rnn: (None, 4, 256)
Dimensions of x1: (None, 4, 256)
Dimensions of x2: (None, 4, 256)
Dimensions of x after concat: (None, 4, 512)
Dimensions of x after rnn: (None, 4, 256)
Dimensions of x1: (None, 4, 256)
Dimensions of x2: (None, 4, 256)
Dimensions of x after concat: (None, 4, 512)
Dimensions of x after rnn: (None, 4, 256)
Dimensions of x1: (None, 4, 256)
Dimensions of x2: (None, 4, 256)
Dimensions of x after concat: (None, 4, 512)
Dimensions of x after rnn: (None, 4, 256)
Dime

INFO:tensorflow:Assets written to: translator\assets


INFO:tensorflow:Assets written to: translator\assets
